# **Function Calling to Generate JSON data**

##**0. Install Dependencies**

In [26]:
!pip install openai pydantic

##**2. Get API key from Secret**

In [25]:
from google.colab import userdata
# Retrieve the API key from Colab's secrets
api_key = userdata.get('OPENAI_API_KEY')

##**3. Python Code**

In [21]:
from openai import OpenAI

client = OpenAI(api_key=api_key)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": "Extract name, age, and city from the sentence: Ritu Patil is 30 and lives in Pune.",
        }
    ],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "extract_person_info",
                "description": "Extract name, age, and city from text",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"},
                        "age": {"type": "integer"},
                        "city": {"type": "string"},
                    },
                    "required": ["name", "age", "city"],
                },
            },
        }
    ],
    tool_choice="auto",
)

# Extract the structured response
tool_calls = response.choices[0].message.tool_calls
arguments = tool_calls[0].function.arguments

import json
parsed = json.loads(arguments)
print(parsed)

{'name': 'Ritu Patil', 'age': 30, 'city': 'Pune'}


#**Validate using Pydantic**

In [24]:
from pydantic import BaseModel, ValidationError
import json
from openai import OpenAI

# Define your API client
client = OpenAI(api_key=api_key)

# Step 1: Define a Pydantic model
class PersonInfo(BaseModel):
    name: str
    age: int
    city: str

# Step 2: Get the structured output from OpenAI function call
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": "Extract name, age, and city from the sentence: Ritu Patil is 30 and lives in Pune.",
        }
    ],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "extract_person_info",
                "description": "Extract name, age, and city from text",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"},
                        "age": {"type": "integer"},
                        "city": {"type": "string"},
                    },
                    "required": ["name", "age", "city"],
                },
            },
        }
    ],
    tool_choice="auto",
)

# Step 3: Extract and parse tool call arguments
tool_calls = response.choices[0].message.tool_calls
arguments = tool_calls[0].function.arguments
parsed = json.loads(arguments)
print(parsed)

# Step 4: Validate using Pydantic
try:
    validated = PersonInfo(**parsed)
    print("\n✅ Validated:", validated)
except ValidationError as ve:
    print("❌ Validation Error:", ve)


{'name': 'Ritu Patil', 'age': 30, 'city': 'Pune'}

✅ Validated: name='Ritu Patil' age=30 city='Pune'
